# Evaluating Numerical Expressions
In this project,we will use stacks to implement an algorithm that can evaluate numerical expressions. 

In [121]:
from linked_list import LinkedList

In [122]:
class Stack(LinkedList):
    def push(self, data):
        self.append(data)
    def peek(self):
        return self.tail.data
    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

For a computer, it's much easier to evaluate an expression written in postfix notation. In postfix notation, the operands appear before the operator. The infix expression **1 + 2** becomes **1 2 +** in postfix notation.
We can evaluate an expression in postfix notation using a stack. We read the expression from left to right and do the following:

1. If we find a number, we push that number to the top of the stack.
2. If we find an operator, we pop the top two elements of the stack, perform the operation, and then push back the result.

After processing the entire expression, there will be a single element on the stack. This value is the result of the operation.

Let's implement a function *evaluate_postfix()* that evaluates an expression in postfix notation. To simplify the function, we will assume that, in the expression string we want to evaluate, there are spaces between all elements of the expression. Based on this assumption, we can transform the postfix expression string into a list of elements using the **str.split()** method, like so:
**expression = "12 2 4 + / 21 *"
 elements = expression.split()
 print(elements)**

In the context of evaluating expressions, we call these elements **tokens**, and the term for transforming the expression into a list of tokens is **tokenize**. 

In [123]:
# function that tokenizes a postfix expression
def tokenize(expression):
    tokens = expression.split( )
    return tokens
print(tokenize("12 2 4 + / 21 *"))

['12', '2', '4', '+', '/', '21', '*']


There is one important detail we need to consider in the second step. When we find an operator, we pop the top two values on the top of the stack. When we apply the operator to those two elements, we need to make sure we operate those two numbers in the correct order.

Consider the expression **1 - 2**. We need to subtract 2 from 1 and not the other way around. The element at the top of the stack is actually the second operand in the expression, while the element that is second to top is the first operand. 

In [124]:
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)

In [125]:
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)

In [126]:
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)

In [127]:
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)

In [128]:
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

# Evaluating Postfix Expressions
We can now implement an algorithm to evaluate an expression in postfix notation. To do so, we need to do the following:

1. Tokenize the expression using the tokenize() function
2. Initialize an empty stack
3. For each token, do the following:
    1.1. If the token is an operator, call the corresponding function to process it. For example, if we find a +, we call the process_plus() function.
    1.2. Otherwise (the token is a number) we push that number to the top of the stack. Since each token is a string, we'll need to convert it to a float first.
4. Return the value that is left in the stack.

In [129]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token == "+":
            process_plus(stack)
        elif token == "-":
            process_minus(stack)
        elif token == "*":
            process_times(stack)
        elif token == "/":
            process_divide(stack)
        elif token == "**":
            process_pow(stack)
        else:
            # The token is not an operator so it must be a number
            stack.push(float(token))
    return stack.pop()  

In [130]:
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]
for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


# Operator Precedence in Infix Notation

We can now evaluate postfix expressions! But to make this project useful, we need to enable our algorithm to evaluate expressions in infix notation. After all, it would be very annoying to have to write expressions in postfix notation to use our algorithm.

Like before, to simplify tokenizing the expression, we'll assume that the infix expression string contains spaces between any two tokens (even the parentheses).This means that we can tokenize the expression using the **tokenize()** function we implemented before. 

To convert an expression from infix to postfix, we'll implement the <a href="https://en.wikipedia.org/wiki/Shunting_yard_algorithm" target="_blank" rel="noopener">Shunting-yard algorithm</a> . The data structure to implement this algorithm is (again) a stack.

We need to do some prep work before we start describing and implementing the algorithm. In an expression in infix notation, the operation precedence rules define the order in which we perform the operations. For example, in the expression 4 + 2 * 3, we first need to perform the multiplication and only then the addition.

If operators have the same precedence, they are evaluated in the order they appear. For example, in 1 - 2 + 3, we do - and then +, but in 1 + 2 - 3, we do + and then -.

In short, the + and - have the same precedence, the * and / have the same precedence and higher precedence than + and -, and the ** has the highest precedence of all.

In the Shunting-yard algorithm, we'll need to compare the precedence of the operators. We will use numbers to represent the operator precedence. The higher the precedence, the higher the number (+ and - = 1,  * and / = 2, ** = 3).

If we store the precedence table in a dictionary named precedence with keys equal to the operators and values equal to the precedence numbers, we can check whether an operator has lower precedence than another operator.


In [131]:
precedence = {
    "+": 1,
    "-": 1,
    "*": 2,
    "/": 2,
    "**": 3
}

print(precedence["/"] < precedence["-"])
print(precedence["+"] < precedence["*"])
print(precedence["+"] < precedence["-"])
print(precedence["/"] < precedence["**"])

False
True
False
True


# From Infix to Postfix
We'll implement a function **infix_to_postfix()** that, given a string with an expression in infix notation, outputs a string with that expression written in postfix notation. This function will implement the <a href="https://en.wikipedia.org/wiki/Shunting_yard_algorithm" target="_blank" rel="noopener">Shunting-yard algorithm </a>. This algorithm is similar to the **evaluate_postfix()** function we've implemented before. It starts by tokenizing the postfix expression, and then it processes the tokens one by one using a stack. It builds the postfix expression by keeping track of a list named postfix, which will contain the list of tokens in postfix order.

Here's how processing should go for each token:

1. Opening parenthesis, (:
    - Push the token into the stack for later use when we find a closing parenthesis.
2. Closing parenthesis ):
    - While the top of the stack isn't an opening parenthesis, (, pop the top element, and append it to the postfix token list.
    - Pop the opening parentheses out of the stack at the end.
3. Operator, +, -, *, /, or '**': 
    - While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element, and append it to the postfix token list.
    - Push the current operator to the top of the stack
4. Operand (any number):
    - Append the number to the postfix token list.

After having processed all tokens, the stack may not be empty. In this case we pop all remaining operators into the postfix list.

# Opening parenthesis
- Opening parentheses,**(**:
Push the token into the stack. It will be used later when we find a closing parenthesis.

In [132]:
# push the string "(" into the stack
def process_opening_parenthesis(stack):
    stack.push("(")    

# Closing Parentheses
- Closing parenthesis **)**:
1. While the top of the stack isn't an opening parenthesis,**(**, pop the top element, and append it to the **postfix** token list.
2. Pop the opening parentheses out of the stack at the end.

In [133]:
def process_closing_parenthesis(stack, postfix):
#     while the top element of the stack is not an opening parenthesis,
    while stack.peek() != "(":
        postfix.append(stack.pop())
#     pop the top element of the stack(()
    stack.pop()

# Handling Operators
- Operator, +, -, *, / or **:
1. While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element and append it to the postfix token list.
2. Push the current operator to the top of the stack.

Here, we see again that we need to use a while loop. The condition of the while loop needs to check that the top of the stack is an operator and that its precedence is greater than or equal to the precedence of the operator we're processing.

We can get the top of the stack (without removing it) using the Stack.peek() function. However, we first need to ensure that the stack isn't empty, or it will cause an error.

Earlier, we defined a dictionary named precedence to compare the precedence of two operators. 

In [134]:
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
    # Pop the top of the stack into the postfix list
        postfix.append(stack.pop())
#     push the operator into the stack
    stack.push(operator)

# Handling Numbers
- Operand (any number):
1. Push the token into the postfix token list.

In [135]:
def process_number(postfix, number):
    postfix.append(number)

# Implementing the Shunting-yard Algorithm

We now have all the pieces we need to implement the infix_to_postfix() function that converts an expression from infix notation to postfix notation.

This function will work as follows:

1. We start by splitting the expression into tokens using the tokenize() function.
2. We initialize an empty stack.
3. We initialize an empty postfix token list.
4. Iterate over all tokens, and for each, do the following:
    - If the token is "(", we call the process_opening_parenthesis() function.
    - If the token is ")", we call the process_closing_parenthesis() function.
    - If the token is an operator, we call the process_operator() function.
    - Otherwise, the token is a number, and we call the process_number() function.
5. After processing all tokens, we use a while loop to pop the remaining stack element into the postfix token list.
6. Use the str.join() method to convert the postfix token list into a string.

In [136]:
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack) > 0:
        postfix.append(stack.pop())
    return " ".join(postfix)

# Evaluating Infix expressions
We now have a function that can transform an infix expression into postfix notation and a function that can evaluate an expression in postfix notation. By combining the two, we can write a function named **evaluate()** that returns the value of an expression in infix notation.

In [137]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

In [138]:
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions:
    print(evaluate(expression))

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0
